In [1]:
import pandas as pd
import numpy as np

In [2]:
data_preprocessed = pd.read_csv('Absenteeism_preprocessed.csv')

In [3]:
data_preprocessed.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,False,False,False,True,7,1,289,36,33,239.554,30,0,2,1,4
1,False,False,False,False,7,1,118,13,50,239.554,31,0,1,0,0
2,False,False,False,True,7,2,179,51,38,239.554,31,0,0,0,2
3,True,False,False,False,7,3,279,5,39,239.554,24,0,2,0,4
4,False,False,False,True,7,3,289,36,33,239.554,30,0,2,1,2


In [4]:
data_preprocessed['Absenteeism Time in Hours'].median()

3.0

In [5]:
#we classified everyone with absenteeism time in hours <= 3 as moderately absent, set them as 0
#we classified everyone with absenteeism time in hours > 3 as excessively absent, set them as 1

### targets

In [6]:
#create targets
targets = np.where(data_preprocessed['Absenteeism Time in Hours'] > 
                   data_preprocessed['Absenteeism Time in Hours'].median(), 1, 0)

In [7]:
data_preprocessed['Excessive Absenteeism'] = targets

In [8]:
data_preprocessed.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Excessive Absenteeism
0,False,False,False,True,7,1,289,36,33,239.554,30,0,2,1,4,1
1,False,False,False,False,7,1,118,13,50,239.554,31,0,1,0,0,0
2,False,False,False,True,7,2,179,51,38,239.554,31,0,0,0,2,0
3,True,False,False,False,7,3,279,5,39,239.554,24,0,2,0,4,1
4,False,False,False,True,7,3,289,36,33,239.554,30,0,2,1,2,0


In [9]:
#find the percentage of excessive absenteeism
targets.sum() / targets.shape[0]

0.45571428571428574

In [10]:
#shows that around 45% of targets are excessive absenteeism
#thus, around 55% of targets aee moderately absenteeism

In [11]:
data_with_targets = data_preprocessed.drop(['Absenteeism Time in Hours','Day of the week',
                                            'Daily Work Load Average','Distance to Work'],axis=1)

In [12]:
#check if two df are different
data_with_targets is data_preprocessed

False

### select the inputs

In [13]:
#check shape
data_with_targets.shape

(700, 12)

In [14]:
data_with_targets.head(1)

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Excessive Absenteeism
0,False,False,False,True,7,289,33,30,0,2,1,1


In [15]:
#select every row, every column except Excessive Absenteeism since it's a target
#iloc[:, :-1] = every row, every column except the last column
data_with_targets.iloc[:, :-1]

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets
0,False,False,False,True,7,289,33,30,0,2,1
1,False,False,False,False,7,118,50,31,0,1,0
2,False,False,False,True,7,179,38,31,0,0,0
3,True,False,False,False,7,279,39,24,0,2,0
4,False,False,False,True,7,289,33,30,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...
695,True,False,False,False,5,179,40,22,1,2,0
696,True,False,False,False,5,225,28,24,0,1,2
697,True,False,False,False,5,330,28,25,1,0,0
698,False,False,False,True,5,235,32,25,1,0,0


In [16]:
unscaled_inputs = data_with_targets.iloc[:, :-1]

In [17]:
#from sklearn.preprocessing import StandardScaler

In [18]:
#create empty standard scaler object
#absenteeism_scaler = StandardScaler()

In [19]:
#scale only value we choose(not scale dumies)
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

# create the Custom Scaler class

class CustomScaler(BaseEstimator,TransformerMixin): 
    
    # init or what information we need to declare a CustomScaler object
    # and what is calculated/declared as we do
    
    def __init__(self,columns):
        
        # scaler is nothing but a Standard Scaler object
        self.scaler = StandardScaler()
        # with some columns 'twist'
        self.columns = columns
        self.mean_ = None
        self.var_ = None
        
    
    # the fit method, which, again based on StandardScale
    
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self
    
    # the transform method which does the actual scaling

    def transform(self, X, y=None, copy=None):
        
        # record the initial order of the columns
        init_col_order = X.columns
        
        # scale all features that you chose when creating the instance of the class
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        
        # declare a variable containing all information that was not scaled
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        
        # return a data frame which contains all scaled features and all 'not scaled' features
        # use the original order (that you recorded in the beginning)
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [20]:
unscaled_inputs.columns.values

array(['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month Value',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object)

In [21]:
#columns_to_scale = ['Month Value',
 #      'Day of the week', 'Transportation Expense', 'Distance to Work',
  #     'Age', 'Daily Work Load Average', 'Body Mass Index', 'Education',
   #    'Children', 'Pets']
columns_to_omit = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4','Education']

In [22]:
columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_to_omit]

In [23]:
absenteeism_scaler = CustomScaler(columns_to_scale)

In [24]:
#prepare scaling mechanism
absenteeism_scaler.fit(unscaled_inputs)

CustomScaler(columns=['Month Value', 'Transportation Expense', 'Age',
                      'Body Mass Index', 'Children', 'Pets'])

In [25]:
#scale the inputs
scaled_inputs = absenteeism_scaler.transform(unscaled_inputs)

In [26]:
scaled_inputs

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets
0,False,False,False,True,0.182726,1.005844,-0.536062,0.767431,0,0.880469,0.268487
1,False,False,False,False,0.182726,-1.574681,2.130803,1.002633,0,-0.019280,-0.589690
2,False,False,False,True,0.182726,-0.654143,0.248310,1.002633,0,-0.919030,-0.589690
3,True,False,False,False,0.182726,0.854936,0.405184,-0.643782,0,0.880469,-0.589690
4,False,False,False,True,0.182726,1.005844,-0.536062,0.767431,0,0.880469,0.268487
...,...,...,...,...,...,...,...,...,...,...,...
695,True,False,False,False,-0.388293,-0.654143,0.562059,-1.114186,1,0.880469,-0.589690
696,True,False,False,False,-0.388293,0.040034,-1.320435,-0.643782,0,-0.019280,1.126663
697,True,False,False,False,-0.388293,1.624567,-1.320435,-0.408580,1,-0.919030,-0.589690
698,False,False,False,True,-0.388293,0.190942,-0.692937,-0.408580,1,-0.919030,-0.589690


In [27]:
scaled_inputs.shape

(700, 11)

### split the data into train and test

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
#split the data into 
#train dataset with input, 
#train dataset with target, 
#test dataset with input, 
#test data set with target
train_test_split(scaled_inputs, targets)

[     Reason_1  Reason_2  Reason_3  Reason_4  Month Value  \
 217      True     False     False     False    -0.388293   
 639     False     False     False      True    -0.959313   
 660      True     False     False     False    -0.673803   
 119     False     False     False      True    -1.530333   
 94      False     False     False      True     1.324766   
 ..        ...       ...       ...       ...          ...   
 646     False     False     False      True    -0.959313   
 478     False     False     False      True     0.182726   
 634      True     False     False     False    -0.959313   
 187      True     False     False     False    -0.673803   
 205     False     False     False      True    -0.388293   
 
      Transportation Expense       Age  Body Mass Index  Education  Children  \
 217               -0.654143  0.248310         1.002633          0 -0.919030   
 639                1.005844 -0.536062         0.767431          0  0.880469   
 660                2.2131

In [30]:
#declair the result of split 
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, 
                                                    targets, 
                                                    train_size = 0.8, #to set the size of train and test data(80/20 in this case)
                                                    random_state = 666) #makes the method psudo random the data set

In [31]:
#check shapes
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(560, 11) (560,)
(140, 11) (140,)


In [32]:
print("Train data:")
print("  Input shape =", x_train.shape)
print("  Target shape =", y_train.shape)

print("Test data:")
print("  Input shape =", x_test.shape)
print("  Target shape =", y_test.shape)

Train data:
  Input shape = (560, 11)
  Target shape = (560,)
Test data:
  Input shape = (140, 11)
  Target shape = (140,)


### logictic regression with sklearn

In [33]:
from sklearn.linear_model import LogisticRegression

In [34]:
from sklearn import metrics

### train the model

In [35]:
reg = LogisticRegression()

In [36]:
reg.fit(x_train, y_train)

LogisticRegression()

In [37]:
#check accuracy of the model via .score method
reg.score(x_train, y_train)

0.7803571428571429

### manually check accuracy

In [38]:
model_outputs = reg.predict(x_train)

In [39]:
model_outputs

array([1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1,

In [40]:
#check if the result of the prediction from predict method and from the target is the same
model_outputs == y_train

array([ True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False, False,  True,  True,  True,  True, False,
        True,  True,  True,  True, False, False,  True,  True,  True,
       False,  True, False,  True, False, False,  True, False,  True,
        True,  True, False,  True,  True,  True,  True,  True, False,
       False, False, False,  True,  True, False,  True, False, False,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True, False, False,  True,
        True,  True,  True, False,  True,  True,  True,  True, False,
        True,  True, False,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [41]:
#check how many correct prediction
sum(model_outputs == y_train)

437

In [42]:
#check total observation
model_outputs.shape[0]

560

In [43]:
#manual accuracy check
sum(model_outputs == y_train)/model_outputs.shape[0]

0.7803571428571429

### find intercept and coefficient 

In [44]:
reg.intercept_

array([-1.67438256])

In [45]:
reg.coef_

array([[ 3.07890932,  0.60182057,  2.93635714,  0.78784663,  0.17418071,
         0.60133577, -0.17141879,  0.25960452, -0.07312492,  0.43619216,
        -0.27021357]])

In [46]:
feature_name = unscaled_inputs.columns.values

In [47]:
#create summary table including intercept and coefficient
summary_table = pd.DataFrame(columns = ['Feature_name'], data = feature_name)

In [48]:
summary_table['Coefficient'] = np.transpose(reg.coef_)

In [49]:
summary_table.index = summary_table.index + 1

In [50]:
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]

In [51]:
summary_table = summary_table.sort_index()

In [52]:
summary_table
#coeff also known as weight
#intercept also known as bias

,Feature_name,Coefficient
0,Intercept,-1.674383
1,Reason_1,3.078909
2,Reason_2,0.601821
3,Reason_3,2.936357
4,Reason_4,0.787847
5,Month Value,0.174181
6,Transportation Expense,0.601336
7,Age,-0.171419
8,Body Mass Index,0.259605
9,Education,-0.073125


In [53]:
#add exponential of coefficient as a new column and named it Odds_ratio
summary_table['Odds_ratio'] = np.exp(summary_table.Coefficient)

In [54]:
#interpret the importance of features by their odd ratio
summary_table.sort_values('Odds_ratio', ascending = False)
#the bigger the coeff(no matter + or -) from 0 is, the bigger the feature

,Feature_name,Coefficient,Odds_ratio
1,Reason_1,3.078909,21.734684
3,Reason_3,2.936357,18.847064
4,Reason_4,0.787847,2.198657
2,Reason_2,0.601821,1.825439
6,Transportation Expense,0.601336,1.824554
10,Children,0.436192,1.546806
8,Body Mass Index,0.259605,1.296417
5,Month Value,0.174181,1.190271
9,Education,-0.073125,0.929485
7,Age,-0.171419,0.842469


In [55]:
#based on the result of interpretation, we backward elimination to drop distance to work, daily work load and education

### test model

In [56]:
reg.score(x_test, y_test)

0.6642857142857143

In [57]:
predict_proba = reg.predict_proba(x_test)

In [58]:
predict_proba #probability of x test being 0 and 1

array([[0.76019338, 0.23980662],
       [0.24626232, 0.75373768],
       [0.52249881, 0.47750119],
       [0.78106376, 0.21893624],
       [0.83972713, 0.16027287],
       [0.05268042, 0.94731958],
       [0.32136172, 0.67863828],
       [0.67128565, 0.32871435],
       [0.55511701, 0.44488299],
       [0.80550567, 0.19449433],
       [0.849804  , 0.150196  ],
       [0.74515846, 0.25484154],
       [0.83292056, 0.16707944],
       [0.87346907, 0.12653093],
       [0.43453351, 0.56546649],
       [0.34342678, 0.65657322],
       [0.13291364, 0.86708636],
       [0.33458796, 0.66541204],
       [0.79257824, 0.20742176],
       [0.20489315, 0.79510685],
       [0.50781392, 0.49218608],
       [0.29181275, 0.70818725],
       [0.62780815, 0.37219185],
       [0.27724969, 0.72275031],
       [0.87675427, 0.12324573],
       [0.76561717, 0.23438283],
       [0.87043956, 0.12956044],
       [0.878864  , 0.121136  ],
       [0.8206162 , 0.1793838 ],
       [0.19691029, 0.80308971],
       [0.

In [59]:
predict_proba[:, 1] #slice the probability of excessive absenteeism

array([0.23980662, 0.75373768, 0.47750119, 0.21893624, 0.16027287,
       0.94731958, 0.67863828, 0.32871435, 0.44488299, 0.19449433,
       0.150196  , 0.25484154, 0.16707944, 0.12653093, 0.56546649,
       0.65657322, 0.86708636, 0.66541204, 0.20742176, 0.79510685,
       0.49218608, 0.70818725, 0.37219185, 0.72275031, 0.12324573,
       0.23438283, 0.12956044, 0.121136  , 0.1793838 , 0.80308971,
       0.24342439, 0.66769774, 0.45568335, 0.7326042 , 0.94494273,
       0.82148655, 0.09278469, 0.48196366, 0.28189628, 0.4095337 ,
       0.12324573, 0.90966363, 0.8471343 , 0.2005841 , 0.2005841 ,
       0.18819613, 0.18682142, 0.72501108, 0.1959119 , 0.90193271,
       0.21577102, 0.82148655, 0.45656206, 0.11797195, 0.778502  ,
       0.49286498, 0.22430495, 0.90170661, 0.18682142, 0.6244003 ,
       0.2170082 , 0.90193271, 0.22064453, 0.79510685, 0.12324573,
       0.48991994, 0.64224769, 0.7348141 , 0.28473363, 0.68204601,
       0.24551353, 0.11797195, 0.35102313, 0.97323285, 0.16707

### save the model

In [60]:
import pickle

In [61]:
with open('model', 'wb') as file:
    pickle.dump(reg, file)

In [62]:
with open('scaler','wb') as file:
    pickle.dump(absenteeism_scaler, file)